In [ ]:
# ============================================================
# Monte Carlo Simulation of Diffusion-Reaction in Porous Media
# Based on: Guo & Keil (2003), Coppens (1999)
# Final Clean Version – Suitable for Reproducibility / Supplementary Code
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
from numba import njit

# ------------------------------------------------------------
# Config (모든 실험 파라미터 관리)
# ------------------------------------------------------------
params = {
    "seed": 0,
    "level": 3,
    "k_keep_list": [12, 14, 16, 18, 20, 22, 24],  # fractal generator
    "Pr_list": np.geomspace(5e-3, 1.0, 10),        # reaction probability sweep
    "Ndif_list": [1, 10],                          # Knudsen diffusion steps
    "mcs_total": 1200,
    "mcs_burn": 600,
    "n_atm": 50
}

rng = np.random.default_rng(params["seed"])

# ------------------------------------------------------------
# Geometry Builders
# ------------------------------------------------------------
def make_generator_mask(k_keep: int):
    coords = []
    for z in range(3):
        for y in range(3):
            for x in range(3):
                m = abs(x-1)+abs(y-1)+abs(z-1)
                coords.append(((z, y, x), m))
    coords.sort(key=lambda t: t[1], reverse=True)
    gen = np.zeros((3,3,3), dtype=bool)
    for i in range(k_keep):
        (z,y,x),_ = coords[i]
        gen[z,y,x] = True
    return gen

def build_menger(level=3, k_keep=20):
    solid = np.ones((1,1,1), dtype=bool)
    gen = make_generator_mask(k_keep)
    for _ in range(level):
        solid = np.kron(solid, gen)
    return solid  # True=solid, False=pore

def build_uniform(level=3):
    n = 3**level
    solid = np.ones((n,n,n), dtype=bool)
    w = max(2, n//12)
    solid[:, :, n//2-w:n//2+w] = False
    solid[:, n//2-w:n//2+w, :] = False
    solid[n//2-w:n//2+w, :, :] = False
    return solid

# ------------------------------------------------------------
# Masks
# ------------------------------------------------------------
NB6 = np.array([[1,0,0],[-1,0,0],[0,1,0],[0,-1,0],[0,0,1],[0,0,-1]], dtype=int)

def masks_from_solid(solid: np.ndarray):
    pore = ~solid
    Z,Y,X = solid.shape
    pad = np.pad(solid, 1, constant_values=False)
    surf = np.zeros_like(solid, dtype=bool)
    for dz,dy,dx in NB6:
        neigh = pad[1+dz:1+dz+Z, 1+dy:1+dy+Y, 1+dx:1+dx+X]
        surf |= (solid & (~neigh))
    bpore = np.zeros_like(pore, dtype=bool)
    bpore[0,:,:]  |= pore[0,:,:];   bpore[-1,:,:] |= pore[-1,:,:]
    bpore[:,0,:]  |= pore[:,0,:];   bpore[:,-1,:] |= pore[:,-1,:]
    bpore[:,:,0]  |= pore[:,:,0];   bpore[:,:,-1] |= pore[:,:,-1]
    return pore, surf, bpore

def init_fields(solid: np.ndarray, n_atm=50):
    pore, surf, bpore = masks_from_solid(solid)
    A = np.zeros_like(solid, dtype=np.int32)  # reactant
    P = np.zeros_like(solid, dtype=np.int32)  # product
    A[bpore] = int(n_atm)
    return pore, surf, bpore, A, P

# ------------------------------------------------------------
# Monte Carlo Step (Numba optimized)
# ------------------------------------------------------------
@njit
def mcs_step_numba(solid, pore, bpore, A, P, Ndif=1, Pr=0.1):
    Z,Y,X = solid.shape
    intrinsic = 0
    apparent  = 0

    idx = np.argwhere(pore)
    np.random.shuffle(idx)

    for z,y,x in idx:
        occ = A[z,y,x] + P[z,y,x]
        if occ <= 0:
            continue

        # 1) Choose molecule
        is_A = (np.random.randint(occ) < A[z,y,x])

        # 2) Choose direction
        dz,dy,dx = NB6[np.random.randint(6)]

        # 3) Walk Ndif steps
        zn,yn,xn = z,y,x
        steps = 0
        while steps < Ndif:
            z2,y2,x2 = zn+dz, yn+dy, xn+dx
            if not (0<=z2<Z and 0<=y2<Y and 0<=x2<X): break
            if solid[z2,y2,x2]: break
            zn,yn,xn = z2,y2,x2
            steps += 1

        # 4) Acceptance probability
        n_src = A[z,y,x] + P[z,y,x]
        n_dst = A[zn,yn,xn] + P[zn,yn,xn]
        Pd = np.exp(-(n_dst - n_src))
        if Pd > 1.0: Pd = 1.0
        if np.random.random() < Pd:
            if is_A:
                A[z,y,x] -= 1; A[zn,yn,xn] += 1
            else:
                P[z,y,x] -= 1; P[zn,yn,xn] += 1
            z,y,x = zn,yn,xn

        # 5) Surface reaction
        if is_A and A[z,y,x] > 0:
            touched = False
            for dz2,dy2,dx2 in NB6:
                zz,yy,xx = z+dz2, y+dy2, x+dx2
                if 0<=zz<Z and 0<=yy<Y and 0<=xx<X and solid[zz,yy,xx]:
                    touched = True; break
            if touched and np.random.random() < Pr:
                A[z,y,x] -= 1; P[z,y,x] += 1
                intrinsic += 1

        # 6) Boundary release
        if bpore[z,y,x] and P[z,y,x] > 0:
            apparent += P[z,y,x]
            P[z,y,x] = 0

    return intrinsic, apparent

def run_sim_numba(solid, Ndif=1, Pr=0.1, mcs_total=1200, mcs_burn=600, n_atm=50):
    pore, surf, bpore, A, P = init_fields(solid, n_atm=n_atm)
    intr_hist=[]; app_hist=[]
    for _ in range(mcs_total):
        intr, app = mcs_step_numba(solid, pore, bpore, A, P, Ndif=Ndif, Pr=Pr)
        intr_hist.append(intr); app_hist.append(app)
    intr_hist = np.asarray(intr_hist); app_hist = np.asarray(app_hist)
    intr_ss = intr_hist[mcs_burn:].mean() if mcs_total>mcs_burn else intr_hist.mean()
    app_ss  = app_hist[mcs_burn:].mean()  if mcs_total>mcs_burn else app_hist.mean()
    return intr_ss, app_ss, intr_hist, app_hist

# ------------------------------------------------------------
# Example Usage: Pr sweep (Uniform vs Fractal)
# ------------------------------------------------------------
def sweep_Pr_curves(struct="fractal", level=3, k_keep=20, Ndif=1, Pr_list=None):
    if Pr_list is None:
        Pr_list = params["Pr_list"]
    rates=[]
    for Pr in Pr_list:
        solid = build_menger(level, k_keep) if struct=="fractal" else build_uniform(level)
        _, app, *_ = run_sim_numba(solid, Ndif=Ndif, Pr=Pr,
                                   mcs_total=params["mcs_total"],
                                   mcs_burn=params["mcs_burn"],
                                   n_atm=params["n_atm"])
        rates.append(app)
    return np.array(Pr_list), np.array(rates)

# ------------------------------------------------------------
# Quick Test
# ------------------------------------------------------------
if __name__ == "__main__":
    Pr, rate = sweep_Pr_curves("fractal", level=3, k_keep=20, Ndif=10)
    plt.loglog(Pr, rate, "o-")
    plt.xlabel("Pr"); plt.ylabel("apparent rate")
    plt.title("Fractal, Ndif=10")
    plt.grid(True)
    plt.show()
